In [1]:
import torch.nn as nn
import torch.nn.functional as F
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv1=nn.Conv2d(1,6,5)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
        
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),2)
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(x.size()[0],-1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

net=LeNet()
print(net)
        

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
print(len(list(net.parameters())))

10


In [3]:
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [4]:
import torch
input=torch.randn(1,1,32,32)
output=net(input)
output.size()

torch.Size([1, 10])

In [5]:
###损失函数
target=torch.randn(1,10)

In [6]:
criterion=nn.MSELoss()   #对于mean square error，output和target的size是相同的
loss=criterion(output,target)
loss

tensor(0.4677, grad_fn=<MseLossBackward>)

In [7]:
net.zero_grad()
print('bp前con1.bias梯度')
print(net.conv1.bias.grad)
loss.backward()
print('bp后con1.bias梯度')
print(net.conv1.bias.grad)


bp前con1.bias梯度
None
bp后con1.bias梯度
tensor([-0.0043,  0.0024, -0.0039, -0.0034,  0.0083, -0.0017])


In [11]:
###优化器###
#手动实现
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)   #inplace 原地减法